In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
from urllib.request import urlopen
import plotly.express as px

In [3]:
#inputs
merge_by_month = pd.read_csv('Final_Merge_7_15_20.csv')
merge_by_county = pd.read_csv('Regression_Mike/Final Merge 7_17_20.csv')
fips_df = pd.read_csv('Resources/county_fips.csv')

In [4]:
by_month = pd.merge(merge_by_month, fips_df, on='County')
by_month = by_month[[
    'County',
    'Political_Party',
    'Household Income Mean',
    'fips',
    'Month',
    'NewPos_All',
    'Deaths',
    'Hospitalizations_hospitalization'
]]

by_month.columns = [
    'County',
    'Political_Party',
    'Household Income Mean',
    'fips',
    'Month',
    'Cases',
    'Deaths',
    'Hospitalizations'
]

In [5]:
february = by_month[by_month['Month'] == 2].groupby(['County', 'Political_Party', 'fips', 'Month']).sum().reset_index()
march = by_month[by_month['Month'] == 3].groupby(['County', 'Political_Party', 'fips', 'Month']).sum().reset_index()
april = by_month[by_month['Month'] == 4].groupby(['County', 'Political_Party', 'fips', 'Month']).sum().reset_index()
may = by_month[by_month['Month'] == 5].groupby(['County', 'Political_Party', 'fips', 'Month']).sum().reset_index()
june = by_month[by_month['Month'] == 6].groupby(['County', 'Political_Party', 'fips', 'Month']).sum().reset_index()

months = [february, march, april, may, june]

In [6]:
february.name = 'february'
march.name = 'march'
april.name = 'april'
may.name = 'may'
june.name = 'june'

In [7]:
by_county = pd.merge(merge_by_county, fips_df, on='County')
by_county = by_county[[
    'County',
    'Political_Party',
    'Household Income Mean',
    'Percent Below Poverty Level',
    'fips',
    'NewPos_All',
    'Deaths',
    'Hospitalizations_hospitalization',
    'Population',
    'Case_per100k',
    'Deaths_per100k',
    'Hoptializations_per100k',
    'Population_Density'
]]

by_county.columns = [
    'County',
    'Political_Party',
    'Household Income Mean',
    'Percent Below Poverty Level',
    'fips',
    'Cases',
    'Deaths',
    'Hospitalizations',
    'Population',
    'Cases per 100k',
    'Deaths per 100k',
    'Hoptializations per 100k',
    'Population Density'
]
#by_county.head()

In [8]:
wash_lat = 47.7511
wash_lon = -120.7401

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [9]:
def state_map(df, metric, color1, color2):
    fig = px.choropleth_mapbox(
        df,
        geojson=counties, 
        locations='fips', 
        color=f'{metric}',
        color_continuous_scale=[f'{color1}', f'{color2}'],
        mapbox_style='carto-positron',
        zoom=5.25,
        center= {"lat": wash_lat, "lon": wash_lon},
        opacity=0.5,
        labels={f'{metric}': f'{metric}'}
    )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.write_image('map_visuals/' + f'{metric}/'f'{df.name}_{metric}.png')

In [44]:
for month in months:
    state_map(month, 'Cases', '#F9E79F', '#9A7D0A')
    state_map(month, 'Deaths', '#E6B0AA', '#7B241C')
    state_map(month, 'Hospitalizations', '#BB8FCE', '#5B2C6F')

In [18]:
def state_map_summary(metric, color1, color2, color3):
    fig = px.choropleth_mapbox(
        by_county,
        geojson=counties, 
        locations='fips', 
        color=f'{metric}',
        color_continuous_scale=[f'{color1}', f'{color2}', f'{color3}'],
        mapbox_style='carto-positron',
        zoom=5.25,
        center= {"lat": wash_lat, "lon": wash_lon},
        opacity=0.75,
        labels={f'{metric}': f'{metric}'}
    )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.write_image('map_visuals/by_county/'f'{metric}.png')

In [20]:
state_map_summary('Population','white','green','blue')
state_map_summary('Cases per 100k','yellow','red','purple')
state_map_summary('Deaths per 100k','blue','yellow','red')
state_map_summary('Hoptializations per 100k','white','pink','purple')
state_map_summary('Percent Below Poverty Level','yellow','orange','red')

In [10]:
fig = px.choropleth_mapbox(
    by_county, 
    geojson=counties, 
    locations='fips', 
    color='Political_Party',
    hover_name='County',
    color_discrete_map={
        'Republican': 'red',
        'Democratic': 'blue'
    },
    #color_continuous_scale='Viridis',
    mapbox_style='carto-positron',
    zoom=5.25,
    center= {"lat": 47.7511, "lon": -120.7401},
    opacity=0.5,
    labels={'Political_Party':'Political_Party'},
    title='Political Affiliation by County'
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_image('map_visuals/politics_map.png')